House Price Prediction: Load Best MLflow Model & Deploy with Gradio
-

In [ ]:
# ===============================
# 1. Install & Import Libraries
# ===============================

!pip install mlflow


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import mlflow
import mlflow.sklearn
import pandas as pd

# ===============================
# 2. MLflow Configuration
# ===============================
mlflow.set_tracking_uri("http://127.0.0.1:5000")

experiment_name = "House_Price_Prediction_Model_Train_Log"
experiment = mlflow.get_experiment_by_name(experiment_name)
exp_id = experiment.experiment_id

print("Experiment ID:", exp_id)

# ===============================
# 3. Get Best Run (by R2)
# ===============================
runs = mlflow.search_runs(experiment_ids=[exp_id])

best_run = runs.sort_values("metrics.R2", ascending=False).iloc[0]

best_run_id = best_run.run_id
model_name = best_run["tags.model_type"]

print("Best Model:", model_name)
print("Run ID:", best_run_id)

print("\nMetrics:")
print("R2  :", best_run["metrics.R2"])
print("RMSE:", best_run["metrics.RMSE"])
print("MAE :", best_run["metrics.MAE"])

Experiment ID: 5
Best Model: RandomForest
Run ID: d81db08cf0564937b2d8774e7094fa1b

Metrics:
R2  : 0.993768932764569
RMSE: 7715330321.025255
MAE : 12020.913767222222


Load Best Model Pipeline
-

In [5]:
logged_model_uri = f"runs:/{best_run_id}/{model_name}"
model = mlflow.sklearn.load_model(logged_model_uri)

print("\nModel loaded successfully from MLflow")


Model loaded successfully from MLflow


Extract Categories for Dropdowns
-

In [9]:
# Access preprocessor inside pipeline
preprocessor = model.named_steps["preprocessor"]

# Access OneHotEncoder
ohe = preprocessor.named_transformers_["cat"]

# IMPORTANT: order depends on cat_cols used during training
city_categories  = ohe.categories_[0].tolist()
state_categories = ohe.categories_[1].tolist()

print("\nNumber of States:", len(state_categories))
print("Number of Cities:", len(city_categories))


Number of States: 13
Number of Cities: 337


 Prediction Function
 -

In [10]:
def predict_house_price(livingArea, beds, baths, city, state):
    input_df = pd.DataFrame([{
        "livingArea in sq.ft": livingArea,
        "beds": beds,
        "baths": baths,
        "city": city,
        "state": state
    }])

    prediction = model.predict(input_df)[0]
    return int(prediction)

Gradio Deployment
-

In [11]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## House Price Prediction App")

    with gr.Row():
        # Living area input
        livingArea = gr.Number(
            label="Living Area (sq.ft)",
            value=560,      # starting value
            minimum=560     # cannot go below 560
        )

        # Bedrooms input as slider
        beds = gr.Slider(
            label="Beds",
            minimum=1,
            maximum=10,
            step=1,
            value=3         # default
        )

        # Bathrooms input as slider
        baths = gr.Slider(
            label="Baths",
            minimum=1,
            maximum=10,
            step=1,
            value=2         # default
        )

    with gr.Row():
        city_input = gr.Dropdown(
            choices=city_categories,
            label="City"
        )
        state_input = gr.Dropdown(
            choices=state_categories,
            label="State"
        )

    price_output = gr.Number(label="Predicted Price ($)")

    predict_btn = gr.Button("Predict")

    predict_btn.click(
        fn=predict_house_price,
        inputs=[livingArea, beds, baths, city_input, state_input],
        outputs=price_output
    )


Launch App
-

In [12]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
